## Introduction
The goal of this notebook is to lay out a simple example of the auction equilibrium calculation

In [1]:
using JuMP
using Ipopt
using DataFrames, CSV
using Parameters
using DifferentialEquations
using Distributions
using NLsolve

In [17]:

function getBoundarySMax_twoitem_riskaverse_scale(alpha_max, gamma, q_b, q_e , c, sigma_sq)
    function ce_fun(svec)
        s_max_var = svec[1]
        return getCE_riskaverse_scale(gamma, alpha_max, s_max_var, q_b, q_e , c, sigma_sq) 
    end
    starting_point = alpha_max*1.01*(c' * q_b)

    s_max_sol = nlsolve(ce_fun, [starting_point]; inplace = false)
    return s_max_sol.zero[1]
end

function getCE_riskaverse_scale(gamma, alpha, s, q_b, q_e , c, sigma_sq)
    b = getBidVecAnalytical_riskaverse_scale( gamma, alpha, s, q_b, q_e , c, sigma_sq)    

    b_min_c = b - (alpha*c);

    ce = (q_b' * b_min_c) - (gamma/2) * ((sigma_sq .* b_min_c)' * b_min_c)
end



function getBidVecAnalytical_riskaverse_scale( gamma, alpha, s, q_b, q_e , c, sigma_sq)
    # Compute the optimal unit bid vector for each score by solving the quadratic optimization problem
    T = length(q_e)
    if sum(sigma_sq) > 0

        m = Model(with_optimizer(Ipopt.Optimizer,print_level=0))

        @variables m begin
        b[i=1:T] >= 0
        end

        @objective( m,
                    Max,
                    (sum((q_b[i] * (b[i] - alpha*c[i]) - (gamma / 2) * sigma_sq[i] * (b[i] - alpha*c[i])^2) for i=1:T ))
                 )

              @constraint(m, sum( b[i] * q_e[i] for i=1:T ) == s)

           optimize!(m)

           b_min = value.(b)
   else
       (maxitem,maxindex) = findmax(q_b./q_e)
       b_min = zeros(T)
       b_min[maxindex] = s/q_e[maxindex]
   end

    return b_min

end

# function getDbDsAnalytical_riskaverse_scale( gamma, alpha, b, q_a, q_e , c, sigma_sq)

function getDbDsAnalytical_riskaverse_scale(gamma, alpha, b, q_b, q_e , c, sigma_sq)
#     %Computes anayltical derivative of the optimal bid function of item t
#     % w.r.t. score s given alpha

    T = length(q_e)
    pos_b = [ifelse(b[t] > 0, 1, 0) for t=1:T]

    if (sum(sigma_sq) > 0)
        denom = 1.0/(sum((q_e[t]^2 * pos_b[t]) / sigma_sq[t] for t in 1:T))
        db_ds = [((q_e[t] * pos_b[t] / sigma_sq[t]) * denom) for t in 1:T]'
    else
        db_ds = [ifelse(b[t] > 0, (1.0/q_e[t]), 0) for t=1:T]'
    end

    return(db_ds)

end

function ode_analytical_twoitem_riskaverse_scale( alpha, s, q_a, q_e, c, gamma, sigma_sq)

    alpha_dist = Uniform(0.5, 2);

#     alpha_dist = Truncated(LogNormal(0, 0.2), 2);
    f(a) = pdf(alpha_dist, a);
    F(a) = cdf(alpha_dist, a);

    b = getBidVecAnalytical_riskaverse_scale( gamma, alpha, s, q_a, q_e , c, sigma_sq)
    db_ds = getDbDsAnalytical_riskaverse_scale( gamma, alpha, b, q_a, q_e , c, sigma_sq)


    b_min_c = b - (alpha*c);

    profit_ce = (q_a' * b_min_c) - (gamma)*0.5 * ((sigma_sq .* b_min_c)' * b_min_c)
    profit_term = exp(gamma*profit_ce) - 1;


    numerator = (f(alpha) ./ (1 - F(alpha) - 0.00000000001));

    denominator = db_ds * ((gamma .* q_a) - (gamma^2 .* sigma_sq .* b_min_c) );
    numerator = numerator * profit_term;

    ds_dc = numerator / denominator; # Note: this should be a scalar
    # println("ds_dc: ", ds_dc)

    return ds_dc

end

function getExPostScore_riskaverse_scale(alpha, s, q_a, q_e, c, gamma, sigma_sq)
    b = getBidVecAnalytical_riskaverse_scale( gamma, alpha, s, q_a, q_e , c, sigma_sq)
    total_cost = b' * q_a
    return(total_cost)
end

function firstOrderStatistic_density(alpha, nbidders = 2)
    alpha_dist = LogNormal(0, 0.2);
    F(a) = cdf(alpha_dist, a);
    f(a) = pdf(alpha_dist, a);

    out = ( nbidders * f(alpha) * ((1-F(alpha))^(nbidders - 1)) )
    return out
end

firstOrderStatistic_density (generic function with 2 methods)

### Instantiate Auction Example
#### Base Auction [Data]

In [18]:
# qa = [10; 5];
# qe = [10; 10];
qa = [12; 40];
qe = [10; 50];
c = [5;10];
sigma_sq = [3, 5];
gamma = 1.5;

alpha_max = 2;
alpha_min = 0.5;
s_max = qe'*c*alpha_max

alpha_span = (alpha_max,alpha_min);

# %Boundary Condition for worst type
s_boundary_score = s_max;

# % Initial score value
s0 = s_boundary_score
s0

1100

In [19]:
smax_riskaverse_scale = getBoundarySMax_twoitem_riskaverse_scale(alpha_max, gamma, qa, qe , c, sigma_sq)

ode_analytical_twoitem_riskaverse_scale_subs(s, p, alpha) = ode_analytical_twoitem_riskaverse_scale(alpha, s, qa, qe, c, gamma, sigma_sq)

prob_riskaverse_scale = ODEProblem(ode_analytical_twoitem_riskaverse_scale_subs, smax_riskaverse_scale, alpha_span)


ODEProblem with uType Float64 and tType Float64. In-place: false
timespan: (2.0, 0.5)
u0: 1097.9867070962464

In [20]:
sol_riskaverse_scale = DifferentialEquations.solve(prob_riskaverse_scale,alg_hints=[:stiff])


MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{DiffEqDiffTools.TimeDerivativeWrapper{ODEFunction{false,typeof(ode_analytical_twoitem_riskaverse_scale_subs),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing},Float64,DiffEqBase.NullParameters},Float64},Float64,1})
Closest candidates are:
  Float64(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:200
  Float64(::T) where T<:Number at boot.jl:718
  Float64(!Matched::Int8) at float.jl:60
  ...

In [21]:
# smax_riskaverse_scale = getBoundarySMax_twoitem_riskaverse_scale(alpha_max, gamma, qa, qe , c, sigma_sq)

# ode_analytical_twoitem_riskaverse_scale_subs(s, p, alpha) = ode_analytical_twoitem_riskaverse_scale(alpha, s, qa, qe, c, gamma, sigma_sq)
# prob_riskaverse_scale = ODEProblem(ode_analytical_twoitem_riskaverse_scale_subs, smax_riskaverse_scale, alpha_span)
# sol_riskaverse_scale = DifferentialEquations.solve(prob_riskaverse_scale,alg_hints=[:stiff])

# cost_riskaverse_scale = [getExPostScore_riskaverse_scale(a, s, qa, qe, c, gamma, sigma_sq) for (a,s) in zip(sol_riskaverse_scale.t, sol_riskaverse_scale.u)]

# Plots.plot(sol_riskaverse_scale.t, (cost_riskaverse_scale), color="red", label = "Scaling Auction: Risk Averse Bidders")
# Plots.plot!(title="Equilibrium Ex-Post Costs to the DOT")
# Plots.xlabel!("Bidder Types: Alpha")
# Plots.ylabel!("Ex-Post Cost")
# plot!(legend = :topleft)
# # savefig("toy_example/figs/compare_cost_functions.png")
